# FACTS of the FIFA World Cup dataset



We'll take data frfom the file `fifa_world_cup.csv`

In [1]:
#!/usr/bin/env python3
## SFSU, 2022
## Robert Harakaly
## File description:
## GroupProject, Fifa WorldCup
##

import pandas as pd
import numpy as np
from math import *
from collections import Counter

file = "../doc/sport/fifa_world_cup.csv"
data = pd.read_csv(file)

## 1- Who Scored the most Goals ? And 2- Which team have the best goal frequency per games ?



In [2]:
def max_goal(type):
    if type == "sum":
        most_goal = pd.DataFrame(data.groupby(['home_team']).sum())
        most_goal_a = pd.DataFrame(data.groupby(['away_team']).sum())
    if type == "mean":
        most_goal = pd.DataFrame(data.groupby(['home_team']).mean())
        most_goal_a = pd.DataFrame(data.groupby(['away_team']).mean())
    most_goal = pd.DataFrame(most_goal.iloc[0:, [4]]).to_numpy()
    most_goal_a = pd.DataFrame(most_goal_a.iloc[0:, [5]]).to_numpy()
    tot = []
    for idx, i in enumerate(most_goal_a):
        if type == "sum":
            tot.append(int(i+most_goal[idx]))
        if type == "mean":
            tot.append(float((i+most_goal[idx])/2))
    country = data.groupby('home_team')['home_team'].apply(list)
    c = []
    for idx, i in enumerate(country):
        c.append(i[0])
    m_h = max(most_goal)
    m_a = max(most_goal_a)
    m = max(tot)
    j = 0
    j_h = 0
    j_a = 0
    for idx, i in enumerate(c):
        if tot[idx] == m:
            j = idx
        if most_goal[idx] == m_h:
            j_h = idx
        if most_goal_a[idx] == m_a:
            j_a = idx
    if type == "sum":
        print("Team that scored the most goals at Home is:", c[j_h], "with", m_h[0], "goals")
        print("Team that scored the most goals Away is:", c[j_a], "with", m_a[0], "goals")
        print("Team that scored the most goals Overall is:", c[j], "with", m, "goals")
    if type == "mean":
        print("Team that have the highest goals per match at Home is:", c[j_h], "with", m_h[0], "goals")
        print("Team that have the highest goals per match Away is:", c[j_a], "with", m_a[0], "goals")
        print("Team that have the highest goals per match overall is:", c[j], "with", m, "goals")
    return c

def who_scored_the_most_goals():
    c = max_goal("sum")
    print()
    max_goal("mean")
    return c

c = who_scored_the_most_goals()

Team that scored the most goals at Home is: USA with 589 goals
Team that scored the most goals Away is: Brazil with 396 goals
Team that scored the most goals Overall is: Brazil with 954 goals

Team that have the highest goals per match at Home is: New Caledonia with 2.6785714285714284 goals
Team that have the highest goals per match Away is: Vanuatu with 2.314814814814815 goals
Team that have the highest goals per match overall is: New Caledonia with 2.2582046332046333 goals


## 3- Which team have the cleaner sheet ?

In [3]:
def home_clean_sheet(home, team):
    clean_sheet = pd.DataFrame(data[data[home] == 0])
    clean_sheet = clean_sheet[team].to_numpy()
    cs = []
    for i in clean_sheet:
        cs.append(i)
    a = dict(Counter(cs))
    t = max(a, key=a.get)
    if team == 'home_team':
        tot_game = len(data[data['home_team'] == t])
        print("The team who have to most clean sheet at Home is:", t, "with", a[t], "clean sheets")
        print("This is", (a[t]/tot_game)*100, "% clean sheets")
    if team == 'away_team':
        tot_game = len(data[data['away_team'] == t])
        print("The team who have to most clean sheet Away is:", t, "with", a[t], "clean sheets")
        print("This is", (a[t]/tot_game)*100, "% clean sheets")
    return a, cs, tot_game

def clean_sheet(c):
    a_h, cs, tot_game_h = home_clean_sheet("away_team_score", "home_team")
    a_a, cs, tot_game_a = home_clean_sheet("home_team_score", "away_team")
    tot = []
#    tot_game = tot_game_h + tot_game_a
    for idx, i in enumerate(cs):
        try:
            tot.append(a_h[i]+a_a[i])
        except:
            continue
    m = max(tot)
    j = 0
    for idx, i in enumerate(cs):
        if m == tot[idx]:
            j = idx
            break
    tot_game = len(data[(data['home_team'] == cs[j]) | (data['away_team'] == cs[j])])
    print("The team who have to most clean sheet overall is:", cs[j], "with", m, "clean sheets")
    print("This is", (m/tot_game)*100, "% clean sheets")

clean_sheet(c)

The team who have to most clean sheet at Home is: USA with 152 clean sheets
This is 48.40764331210191 % clean sheets
The team who have to most clean sheet Away is: Zambia with 96 clean sheets
This is 39.50617283950617 % clean sheets
The team who have to most clean sheet overall is: Brazil with 229 clean sheets
This is 52.88683602771363 % clean sheets


## 4- Is it an advantage to play at Home ?

In [4]:
def Home_or_Away():
    global data
    print()
    win = (len(data[data['home_team_result'] == 'Win'])/(len(data))*100)
    lose = (len(data[data['home_team_result'] == 'Lose'])/(len(data))*100)
    draw = (len(data[data['home_team_result'] == 'Draw'])/(len(data))*100)
    print("The team at home have:", win, "% chances to win")
    print("The away team have:", lose, "% chances to win")
    print("There is", draw, "% chances to have a draw")

Home_or_Away()


The team at home have: 49.166004765687056 % chances to win
The away team have: 28.305672839764224 % chances to win
There is 22.528322394548724 % chances to have a draw


## 5- Which team won the most games ?

In [5]:
def who_win_the_most(c, team, score):
    global data
    win = []
    for i in c:
        win.append(len(data[(data[team] == i) & (data['home_team_result'] == score)]))
    m = max(win)
    j = 0
    for idx, i in enumerate(win):
        if m == i:
            j = idx
    tot_game = len(data[data[team] == c[j]])
    if team == 'home_team':
        print("The team who won the most of match is:", c[j], "with", m, "victory at Home which is", m/tot_game*100, "% victory")
    if team == 'away_team':
        print("The team who won the most of match is:", c[j], "with", m, "victory Away which is", m/tot_game*100, "% victory")
    return win


def best_country(c):
    win_h = who_win_the_most(c, 'home_team', "Win")
    win_a = who_win_the_most(c, 'away_team', "Lose")
    win = []
    for idx, i in enumerate(win_h):
        win.append(i + win_a[idx])
    m = max(win)
    j = 0
    for idx, i in enumerate(win):
        if m == i:
            j = idx
    tot_game = len(data[(data['home_team'] == c[j]) | (data['away_team'] == c[j])])
    print("The team who won the most of match is:", c[j], "with", m, "victory Overall which is", m/tot_game*100, "% victory")

best_country(c)

The team who won the most of match is: USA with 194 victory at Home which is 61.78343949044586 % victory
The team who won the most of match is: Brazil with 120 victory Away which is 60.0 % victory
The team who won the most of match is: Brazil with 301 victory Overall which is 69.5150115473441 % victory


In [16]:
GroupA = {"Qatar":0, "Ecuador":0, "Senegal":0, "Netherlands":0}
GroupB = {"England":0, "IR Iran":0, "USA":0, "Wales":0}
GroupC = {"Argentina":0, "Saudi Arabia":0, "Mexico":0, "Poland":0}
GroupD = {"Australia":0, "France":0, "Denmark":0, "Tunisia":0}
GroupE = {"Spain":0, "Germany":0, "Japan":0, "Costa Rica":0}
GroupF = {"Belgium":0, "Canada":0, "Morocco":0, "Croatia":0}
GroupG = {"Brazil":0, "Serbia":0, "Switzerland":0, "Cameroon":0}
GroupH = {"Portugal":0, "Ghana":0, "Uruguay":0, "Korea Republic":0}

GROUP = [GroupA, GroupB, GroupC, GroupD, GroupE, GroupF, GroupG, GroupH]

## 6- Does the Fifa Ranking really matters ?

In this part, we want to know if the Fifa Ranking can really give us information who is more likely to win the game.

The outputs can clearly show us that the Ranking can be a good way to see which team will win. We an also see with this code that playing at Home or Away really matters. <br>
`Win HOME: World class against AWAY: Really Good 	 57.99648506151143`<br>
`Lose HOME: World class against AWAY: Really Good 	 19.507908611599298`<br>
`Draw HOME: World class against AWAY: Really Good 	 22.495606326889277`<br><br>
`Win HOME: Really Good against AWAY: World class 	 29.282868525896415`<br>
`Lose HOME: Really Good against AWAY: World class 	 42.03187250996016`<br>
`Draw HOME: Really Good against AWAY: World class 	 28.68525896414343`<br><br>
We can see that the World Class team have 19% chances to loose at Home but have 29% to loose Away against a Good team.

In [8]:
def rank(team1 = None, team2 = None):
    r = data.loc[:len(data), ['home_team', 'away_team', 'home_team_fifa_rank', 'away_team_fifa_rank', 'home_team_result']]
    bins = [0, 10, 30, 40, 70, 100, 211]
    ranking = ["World class", "Really Good", "Average", "Bad", "Really Bad", "Worst"]
    r["home ranking"] = pd.cut(x=data['home_team_fifa_rank'], bins=bins, labels=ranking)
    r["away ranking"] = pd.cut(x=data['away_team_fifa_rank'], bins=bins, labels=ranking)
    rr = r.loc[:len(r), ['home ranking', 'away ranking', 'home_team_result']]
    win_rate = []
    for g1 in ranking:
        for g2 in ranking:
            game = rr[((rr["home ranking"] == g1) & (rr["away ranking"] == g2))]
            nb_games = len(rr[((rr["home ranking"] == g1) & (rr["away ranking"] == g2))])
            for g in ["Win", "Lose", "Draw"]:
                if team1 != None and team2 != None and team1 == g1 and team2 == g2 or team1 == None and team2 == None:
                    win_rate.append(len(game[game["home_team_result"] == g]) / nb_games * 100)
                    print(g, "HOME:", g1, "against AWAY:", g2, "\t", (len(game[game["home_team_result"] == g]) / nb_games * 100))
            if team1 != None and team2 != None and team1 == g1 and team2 == g2:
                return win_rate
            print()

print(rank("World class", "Bad"))
print()
rank()




Win HOME: World class against AWAY: Bad 	 74.3142144638404
Lose HOME: World class against AWAY: Bad 	 7.4812967581047385
Draw HOME: World class against AWAY: Bad 	 18.204488778054863
[74.3142144638404, 7.4812967581047385, 18.204488778054863]

Win HOME: World class against AWAY: World class 	 43.626062322946176
Lose HOME: World class against AWAY: World class 	 32.01133144475921
Draw HOME: World class against AWAY: World class 	 24.362606232294617

Win HOME: World class against AWAY: Really Good 	 57.99648506151143
Lose HOME: World class against AWAY: Really Good 	 19.507908611599298
Draw HOME: World class against AWAY: Really Good 	 22.495606326889277

Win HOME: World class against AWAY: Average 	 66.66666666666666
Lose HOME: World class against AWAY: Average 	 12.444444444444445
Draw HOME: World class against AWAY: Average 	 20.88888888888889

Win HOME: World class against AWAY: Bad 	 74.3142144638404
Lose HOME: World class against AWAY: Bad 	 7.4812967581047385
Draw HOME: World cl

## 7- Win accruacy for all teams

In [21]:
def rank2(country, team2 = None):
    r = data.loc[:len(data), ['home_team', 'away_team', 'home_team_fifa_rank', 'away_team_fifa_rank', 'home_team_result']]
    bins = [0, 10, 30, 40, 70, 100, 211]
    ranking = ["World class", "Really Good", "Average", "Bad", "Really Bad", "Worst"]
    r["home ranking"] = pd.cut(x=data['home_team_fifa_rank'], bins=bins, labels=ranking)
    r["away ranking"] = pd.cut(x=data['away_team_fifa_rank'], bins=bins, labels=ranking)
    rr = r.loc[:len(r), ['home_team', 'away ranking', 'home_team_result']]
    win_rate = []
    for g2 in ranking:
        game = rr[((rr["home_team"] == country) & (rr["away ranking"] == g2)) & ((rr["away ranking"] == g2) & (rr["home_team"] == country))]
        nb_games = len(rr[((rr["home_team"] == country) & (rr["away ranking"] == g2))])
        for g in ["Win", "Lose", "Draw"]:
            try:
                win_rate.append(len(game[game["home_team_result"] == g]) / nb_games * 100)
                print(g, "HOME:", country, "against AWAY:", g2, "\t", (len(game[game["home_team_result"] == g]) / nb_games * 100))
            except:
                print(g, "HOME:", country, "against AWAY:", g2, "\t", (len(game[game["home_team_result"] == g]) / 1 * 100))
        print()
for G in GROUP:
    for H in list(G.keys()):
       rank2(H)
#rank2("France")

Win HOME: Qatar against AWAY: World class 	 25.0
Lose HOME: Qatar against AWAY: World class 	 75.0
Draw HOME: Qatar against AWAY: World class 	 0.0

Win HOME: Qatar against AWAY: Really Good 	 7.6923076923076925
Lose HOME: Qatar against AWAY: Really Good 	 61.53846153846154
Draw HOME: Qatar against AWAY: Really Good 	 30.76923076923077

Win HOME: Qatar against AWAY: Average 	 9.090909090909092
Lose HOME: Qatar against AWAY: Average 	 90.9090909090909
Draw HOME: Qatar against AWAY: Average 	 0.0

Win HOME: Qatar against AWAY: Bad 	 43.93939393939394
Lose HOME: Qatar against AWAY: Bad 	 22.727272727272727
Draw HOME: Qatar against AWAY: Bad 	 33.33333333333333

Win HOME: Qatar against AWAY: Really Bad 	 50.0
Lose HOME: Qatar against AWAY: Really Bad 	 29.310344827586203
Draw HOME: Qatar against AWAY: Really Bad 	 20.689655172413794

Win HOME: Qatar against AWAY: Worst 	 65.97938144329896
Lose HOME: Qatar against AWAY: Worst 	 11.34020618556701
Draw HOME: Qatar against AWAY: Worst 	 22.680